In [349]:
import pandas as pd 
import numpy as np 
import sys
import pandas_datareader as pdr
import matplotlib.pyplot as plt 
import seaborn as sns 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression 
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error as mse
import itertools
sns.set()

%matplotlib inline

In [ ]:
# loading datasets

tfp = pd.read_csv('TFP.csv', parse_dates = ['year'], index_col = 'year')
comex = pd.read_csv('data_comexstatt.csv', parse_dates = ['date'], index_col = 'date')

CASE 1

In [ ]:
# Taking a quick look at the dataframe

print(tfp.head())
tfp.tail()
tfp.info()

In [ ]:
# calculating summary statistics to get to know the data

summary_statistics = tfp.describe()

print(summary_statistics)

#this line of code gives us summary statistics for all countries in the dataset combined, which isn´t of much help

In [ ]:
# in order to understand the data for each single country, we must pivot the dataframe

pivoted_df = pd.pivot_table(tfp, index = 'year', columns = 'isocode')

In [ ]:
pivoted_df.head(5)

In [ ]:
pivoted_df.columns

In [ ]:
# Slicing the dataframe in order do analyze each country specifically

usa = pivoted_df.loc[:, (slice(None), 'USA')]
mex = pivoted_df.loc[:, (slice(None), 'MEX')]
can = pivoted_df.loc[:, (slice(None), 'CAN')]

In [ ]:
# Calculating summary statistics by country

summary = pivoted_df.describe()

print(summary)

In [ ]:
# Data visualization -- Lineplot

_ = plt.plot(usa, label = 'USA')
_ = plt.plot(mex, label = 'MEX')
_ = plt.plot(can, label = 'CAN')
_ = plt.xlabel('Years')
_ = plt.ylabel('TFP')
_ = plt.title('Total Factor Productivity in USA, Canada and, Mexico')
_ = plt.legend()
plt.show()


In [ ]:
# Construction of boxplots

_ = sns.boxplot(data = pivoted_df)

In [ ]:
# Construction of histograms 

usa.hist()
plt.title('Histogram of USA TFP')
plt.show()

In [ ]:
mex.hist()
plt.title('Histogram os MEX TFP')
plt.show()

In [ ]:
can.hist() 
plt.title('Histogram of CAN TFP')
plt.show()

In [ ]:
# Forecasting USA TFP 

plot_acf(usa, lags = 60)
plot_pacf(usa, lags = 60)
plt.show()


test = adfuller(usa, regression = 'ct', regresults = True)
print('The p-value for the ADF test is', test[1])

# Given that the p-value of the ADF test is 0.85, we cannot reject the null hypothesis that there is a unit root in the series
# We must take the difference 

usa_diff = usa.diff()

usa_diff = usa_diff.dropna()

# Doing the ADF test once again 

test_1 = adfuller(usa_diff, regression = 'ct', regresults = True)
print('The p-value for the ADF test is', test_1[1])

plot_acf(usa_diff, lags = 60)
plot_pacf(usa_diff, lags = 60)

In [ ]:
# Grid Search for optimal paramaters in terms of best AIC score
p = d = q = range(0,5) # p, d, and q can be either 0, 1, 2, 3, or 4
pdq = list(itertools.product(p,d,q)) # gets all possible combinations of p, d, and q
combs = {} # stores aic and order pairs
aics = [] # stores aics
# Grid Search continued
for combination in pdq:
    try:
        model = ARMA(usa_diff, order=combination) # create all possible models
        model = model.fit()
        combs.update({model.aic : combination}) # store combinations
        aics.append(model.aic)
    except:
        continue

best_aic = min(aics)

# Model Creation and Forecasting
model_0 = ARMA(usa_diff, order=combs[best_aic])
model_0 = model_0.fit()
print(model.summary())

In [ ]:
# The Grid Search tells us that the best model por USA PTF is simply the mean. 
# With this, we can conclude that the best forecast will be the mean. 

In [ ]:
# Forecasting MEX TFP 

# Forecasting USA TFP 

plot_acf(mex, lags = 60)
plot_pacf(mex, lags = 60)
plt.show()


test = adfuller(usa, regression = 'ct', regresults = True)
print('The p-value for the ADF test is', test[1])

In [ ]:
# Given that the p-value of the ADF test is 0.85, we cannot reject the null hypothesis that there is a unit root in the series
# We must take the difference 

mex_diff = mex.diff()

mex_diff = mex_diff.dropna()

# Doing the ADF test once again 

test_1 = adfuller(usa_diff, regression = 'ct', regresults = True)
print('The p-value for the ADF test is', test_1[1])

plot_acf(usa_diff, lags = 60)
plot_pacf(usa_diff, lags = 60)

In [ ]:
# Grid Search for optimal paramaters in terms of best AIC score
p = d = q = range(0,5) # p, d, and q can be either 0, 1, 2, 3, or 4
pdq = list(itertools.product(p,d,q)) # gets all possible combinations of p, d, and q
combs = {} # stores aic and order pairs
aics = [] # stores aics
# Grid Search continued
for combination in pdq:
    try:
        model = ARMA(mex_diff, order=combination) # create all possible models
        model = model.fit()
        combs.update({model.aic : combination}) # store combinations
        aics.append(model.aic)
    except:
        continue

best_aic = min(aics)

# Model Creation and Forecasting
model = ARMA(mex_diff, order=combs[best_aic])
model = model.fit()
print(model.summary())

# ARMA(3,4)

In [ ]:
# Predictions for Mexico TFP

X = mex.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

for t in range(len(test)):
	model = ARMA(history, order=(3,4))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
error = mse(test, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.plot(test)
plt.plot(predictions, color='red')
plt.legend()
plt.title('MEXICO TFP FORECAST')
plt.show()

In [ ]:
# Forecasting Canada TFP

plot_acf(can, lags = 60)
plot_pacf(can, lags = 60)
plt.show()


test = adfuller(usa, regression = 'ct', regresults = True)
print('The p-value for the ADF test is', test[1])

can_diff = can.diff()

can_diff = can_diff.dropna()

# Doing the ADF test once again 

test_2 = adfuller(usa_diff, regression = 'ct', regresults = True)
print('The p-value for the ADF test is', test_1[1])

plot_acf(can_diff, lags = 60)
plot_pacf(can_diff, lags = 60)

In [ ]:
# Grid Search for optimal paramaters in terms of best AIC score
p = d = q = range(0,5) # p, d, and q can be either 0, 1, 2, 3, or 4
pdq = list(itertools.product(p,d,q)) # gets all possible combinations of p, d, and q
combs = {} # stores aic and order pairs
aics = [] # stores aics
# Grid Search continued
for combination in pdq:
    try:
        model_can = ARMA(can_diff, order=combination) # create all possible models in range
        model_can = model_can.fit()
        combs.update({model_can.aic : combination}) # store combinations
        aics.append(model_can.aic)
    except:
        continue

best_aic = min(aics)

# Model Creation and Forecasting
model = ARMA(can_diff, order=combs[best_aic])
model = model.fit()
print(model.summary())

# ARMA(3,0)

In [ ]:
V = can.values
size = int(len(V) * 0.66)
train, test = V[0:size], V[size:len(V)]
history_can = [v for v in train]
predictions = list()

for t in range(len(test)):
	model = ARMA(history_can, order=(3,0))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
error = mse(test, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.plot(test)
plt.plot(predictions, color='red')
plt.title('CANADA TFP FORECAST')
plt.show()

In [346]:
# Can you think about another feature that could be helpful in explaining TFP series? Explain.

# Another important feature that could be helpful in explaining TFP series is one that captures an economies institutional framework. 

# Recent economic thinking (but as of the times of Adam Smith aswell) postulates that well developed institutions, such as clear and fair competeition rules, contract enforcement, and a well established rule of law,
# for example, are key economics ingredients to promote long-term economict growth via productivity gains. 

# Well desgined instituions tend to reduce transaction costs, facilitating the functionings of the market mechanism. 

# In this sense, important quantitative variables that capture a countries institutional framework can be found in the World Bank´s Doing Business survey. 

# Things such as easy of getting credit, electricity, paying taxes, trading across borders, enforcing contracts, etc. can be found in the survey.

# All these factors are heavily influenced by the institutional framework of the country and impact long-term productivity and growth. 

CASE 2

In [ ]:
comex.head()

In [ ]:
comex.info()

In [ ]:
comex.describe()

In [ ]:
print(comex.groupby(['date', 'product'])['tons'].agg(['mean']))

In [ ]:
# Show the evolution of total monthly and total annual exports from Brazil (all states and to everywhere) 
#of ‘soybeans’, ‘soybean oil’ and ‘soybean meal’;

comex[(comex['product'] == 'soybeans') & (comex['type'] == 'Export')]['tons'].plot()

In [ ]:
comex[(comex['product'] == 'soybean_oil') & (comex['type'] == 'Export')]['tons'].plot()

In [ ]:
comex[(comex['product'] == 'soybean_oil') & (comex['type'] == 'Export')]['tons'].plot()

In [ ]:
#What are the 3 most important products exported by Brazil in the last 5 years?

comex.loc['2014-01-01':].groupby(['product', 'type'])[['tons', 'usd']].agg('mean')

# In terms of tons, the most important products are soybeans, soybean meal, and corn

In [ ]:
# What are the main routes through which Brazil have been exporting ‘corn’ in the last few years?
# Are there differences in the relative importancem of routes depending on the product?

last_few_years = comex.loc['2014-01-01':]

last_few_years[(last_few_years['product'] == 'corn') & (last_few_years['type'] == 'Export')].groupby('route')['tons'].mean()

# The main routes through which corn has been exported have been River and Sea

In [ ]:
# Are there differences in the relative importance of routes depending on the product?

last_few_years.groupby(['type', 'product', 'route'])['tons'].mean()

# In terms of exports, not really. We can see that all products are mainly exported by Sea or River. Imports, however, show a relative importance
# of routes depending on the product. For example, while corn, wheat, and sugar are mainly imported by sea or river, 
# soybean and its derivatives are mainly imported by ground. 

In [ ]:
# Which countries have been the most important trade partners for Brazil in terms of ‘corn’ and ‘sugar’ in the last 3 years?

# I believe we can measure the importance of a trade partner based on the sum of tons imported or exported with them

# By looking at the results below, we can se that in term of corn, Japan, Vietnam, Egypt, Malaysia and SK are important trading partners

# By looking at further results, we can se that in terms of sugar, China, India, United Arab Emirates, Saudi Arabia and Bangladesh are important partners

last_three_years = comex.loc['2016-01-01']

last_three_years[(last_three_years['product'] == 'corn')].groupby(['country', 'type'])['tons'].agg('sum').sort_values()

In [ ]:
last_three_years[(last_three_years['product'] == 'sugar')].groupby(['country', 'type'])['tons'].agg('sum').sort_values()

In [ ]:
# For each of the products in the dataset, show the 5 most important states in terms of exports?

comex[(comex['type'] == 'Export')].groupby('state')['tons'].mean().sort_values()

# The five most important states em terms of average exports are RS, MT, BA, MA, and PR 

In [ ]:
# Manipulating the comex dataframe so that we can merge it with the covariates DF

In [ ]:
comex.reset_index(inplace = True)

In [ ]:
comex.dtypes

In [ ]:
comex['year'] = comex['date'].str.slice(stop = 4)

In [ ]:
comex.head()

In [ ]:
corn_exports = comex[(comex['product'] == 'corn') & (comex['type'] == 'Export')].groupby('year')['tons'].mean()

In [ ]:
soybean_meal_exports = comex[(comex['product'] == 'soybean_meal') & (comex['type'] == 'Export')].groupby('year')['tons'].mean()

In [ ]:
soybean_meal_exports.head(5)

In [ ]:
soybean_meal_df = pd.DataFrame(soybean_meal_exports)

In [ ]:
corn_df = pd.DataFrame(corn_exports)

In [ ]:
corn_df.head()

In [ ]:
corn_df.reset_index(inplace = True)

corn_df['year'] = corn_df['year'].apply(str)

In [ ]:
soybean_meal_df.reset_index(inplace = True)

soybean_meal_df['year'] = soybean_meal_df['year'].apply(str)

In [ ]:
covariates = pd.read_excel('covariates.xlsx', parse_dates = ['year'], index_col = 'year')

In [ ]:
covariates.reset_index(inplace = True)

In [ ]:
covariates.dtypes

In [ ]:
covariates['year'] = covariates['year'].apply(str)

In [ ]:
covariates['year'] = covariates['year'].str.slice(stop = 4)

In [ ]:
covariates.head(10)

In [ ]:
features = covariates.iloc[18:41]

In [ ]:
corn_cov = corn_df.merge(features, how = 'inner')

In [ ]:
soybean_meal_cov = soybean_meal_df.merge(features, how = 'inner')

In [ ]:
# Forecasting tons of corn

# Based on the other questions, we will choose the following explanatory variables: Japan, Vietnam, Egypt

Y = corn_cov['tons'].values
X = corn_cov[['gdp_egypt', 'gdp_japan', 'gdp_vietnam', 'price_corn', 'gdp_world']].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.50, random_state = 42)


# Instantiating Lasso 

lasso = Lasso(alpha = 0.80)

# Fitting data 

lasso.fit(X_train, y_train)

# Predicting data

y_pred = lasso.predict(X_test)

# Getting preditions 

print(len(X_test))

for i in range(len(X_test)): 
    print('X=%s, Predicted=%s' % (X_test[i], y_pred[i]))

# Predicted values in ten years

In [ ]:
# Forecasting tons of soybean meal 

Y = soybean_meal_cov['tons'].values
X = soybean_meal_cov[['gdp_china', 'gdp_spain', 'gdp_iran', 'price_soybean_meal', 'gdp_world']].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.50, random_state = 42)


# Instantiating Lasso 

lasso = Lasso(alpha = 0.80)

# Fitting data 

lasso.fit(X_train, y_train)

# Predicting data

y_pred = lasso.predict(X_test)

# Getting preditions 

print(len(X_test))

for i in range(len(X_test)): 
    print('X=%s, Predicted=%s' % (X_test[i], y_pred[i]))

# Predicted tons of soybean_meal in 10 years 
